In [443]:
import numpy as np
from itertools import chain

In [456]:
with open('aoc14_input_example.txt') as f:
    lines = f.readlines()
lines = [line.strip().split(' -> ') for line in lines]
lines = [[list(map(int,coord.split(','))) for coord in line] for line in lines]
example_input = lines

with open('aoc14_input.txt') as f:
    lines = f.readlines()
lines = [line.strip().split(' -> ') for line in lines]
lines = [[list(map(int,coord.split(','))) for coord in line] for line in lines]
real_input = lines

In [457]:
def flexible_range(i):
    if i == 0:
        return [0]
    elif i > 0:
        return range(i+1)
    else:
        return range(0,i-1,-1)

In [458]:
def get_coords(puzzle_input):
    all_coords = list(chain.from_iterable(lines))
    min_x = min([coord[0] for coord in all_coords])
    min_y = min([coord[1] for coord in all_coords])
    max_x = max([coord[0] for coord in all_coords])
    max_y = max([coord[1] for coord in all_coords])
    return min_x, max_x, min_y, max_y

In [576]:
class FallingSand:
    def __init__(self,puzzle_input,sand_x,use_floor=False):
        self.puzzle_input = puzzle_input
        self.min_x, self.max_x, self.min_y, self.max_y = get_coords(puzzle_input)
        self.sand_x = sand_x - min_x
        self.current_grid = np.chararray((self.max_y+10,self.max_x-self.min_x+10000),unicode=True)
        self.current_grid[:] = '.'
        self.abyss = False
        self.topped = False
        self.use_floor = use_floor
    
    def generate_rock_coords(self):
        all_coords = []
        for line in self.puzzle_input:
            for i in range(len(line)-1):
                x0, y0 = line[i]
                x1, y1 = line[i+1]
                x_diff, y_diff =  x1-x0, y1-y0
                xs = flexible_range(x_diff)
                ys = flexible_range(y_diff)
                for x in xs:
                    for y in ys:
                        all_coords.append([x0+x,y0+y])
        return all_coords

    def place_rocks(self):
        all_coords = generate_rock_coords(self.puzzle_input)
        for coord in all_coords:
            i,j = np.array(coord) - [min_x,0]
            self.current_grid[j][i] = '#'
        if self.use_floor:
            for i in range(0,self.max_x-self.min_x+10000):
                self.current_grid[self.max_y+2][i] = '#'
            
    
    def sand_step(self,position):
        i,j  = position
        if self.current_grid[0][self.sand_x] == 'o':
            self.topped = True
            landed_status = False
        if self.current_grid[j+1][i] == '.':
            landed_status = False
            self.current_grid[j,i] = '.'
            self.current_grid[j+1][i] = '+'
            position = i, j+1
        elif self.current_grid[j+1][i-1] == '.':
            landed_status = False
            self.current_grid[j,i] = '.'
            self.current_grid[j+1][i-1] = '+'
            position = i-1, j+1
        elif self.current_grid[j+1][i+1] == '.':
            landed_status = False
            self.current_grid[j,i] = '.'
            self.current_grid[j+1][i+1] = '+'
            position = i+1, j+1
        else:
            landed_status = True
            self.current_grid[j,i] = 'o'
            position = i, j
        return landed_status, position

    def drop_sand(self):
        landed_status = False
        position = self.sand_x, 0
        while not landed_status:
            landed_status, position = self.sand_step(position)
#             print(max_y, position[1])
            if (position[1] >= self.max_y) and self.use_floor == False:
                print('Abyss!')
                self.abyss = True
                break

In [479]:
sandgame = FallingSand(example_input,500)


In [480]:
sandgame.place_rocks()
sandgame.current_grid

chararray([['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.'],
           ...,
           ['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [481]:
num_drops = 0
while sandgame.abyss == False:
    sandgame.drop_sand()
    num_drops += 1
print(num_drops-1)
sandgame.current_grid


Abyss!
24


chararray([['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.'],
           ...,
           ['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.'],
           ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [547]:
np.savetxt('aoc14_output_example.txt',sandgame.current_grid,fmt='%s')

In [577]:
sandgame = FallingSand(real_input,500,use_floor=True)

In [578]:
sandgame.place_rocks()
sandgame.current_grid
np.savetxt('aoc14_output.txt',sandgame.current_grid,fmt='%s')

In [572]:
num_drops = 0
while sandgame.abyss == False:
    sandgame.drop_sand()
#     print(sandgame.current_grid)
    num_drops += 1
print(num_drops-1)
np.savetxt('aoc14_output.txt',sandgame.current_grid,fmt='%s')

KeyboardInterrupt: 

## Part 2

In [540]:
for i in range(1000):
    sandgame.drop_sand()
    np.savetxt('aoc14_output.txt',sandgame.current_grid,fmt='%s')

In [579]:
num_drops = 0
while sandgame.topped == False:
    sandgame.drop_sand()
#     print(sandgame.current_grid)
    num_drops += 1
    if num_drops % 10000 == 0:
        print(num_drops)
        np.savetxt('aoc14_output.txt',sandgame.current_grid,fmt='%s')
print(num_drops-1)
np.savetxt('aoc14_output.txt',sandgame.current_grid,fmt='%s')

10000
20000
25055
